# hmmlearn框架说明

hmmlearn中主要有两种模型，分布为：GaussianHMM和MultinomialHMM；如果观测值是连续的，那么建议使用GaussianHMM，否则使用MultinomialHMM；参数：初始的隐藏状态概率π参数为: startprob_；状态转移矩阵A参数为: transmat_; 状态和观测值之间的转移矩阵B参数为: emissionprob_(MultinomialHMM模型中)或者在GaussianHMM模型中直接给定均值(means)和方差/协方差矩阵(covars)。

In [4]:
## MultinomialHMM案例
import numpy as np
import hmmlearn.hmm as hmm


# 定义变量
states = ['盒子1', '盒子2', '盒子3']
obs = ['白球', '黑球']
n_states = len(states)
m_obs = len(obs)
start_probability = np.array([0.2, 0.5, 0.3])
transition_probability = np.array([
    [0.5, 0.4, 0.1],
    [0.2, 0.2, 0.6],
    [0.2, 0.5, 0.3]
])
emission_probalitity = np.array([
    [0.4, 0.6],
    [0.8, 0.2],
    [0.5, 0.5]
])

# 定义模型
model = hmm.MultinomialHMM(n_components=n_states)

# 模型属性设置，模型的属性要不通过fit来训练训练数据得到，要不直接给定模型参数
model.startprob_ = start_probability
model.transmat_ = transition_probability
model.emissionprob_ = emission_probalitity


# 运行viterbi预测的问题
se = np.array([
    [0, 1, 0, 0, 1] # 白，黑，白，白，黑
]).T
logprod, box_index = model.decode(se, algorithm='viterbi')
print("颜色:", end="")
print(" ".join(map(lambda t: obs[t], [0, 1, 0, 0, 1])))
print("盒子:", end="")
print(" ".join(map(lambda t: states[t], box_index)))
print("概率值:", end="")
print(np.exp(logprod)) # 这个是因为在hmmlearn底层将概率进行了对数化，防止出现乘积为0的情况

ImportError: cannot import name 'distribute_covar_matrix_to_match_covariance_type'

In [8]:
# 使用MultinomialHMM进行参数的训练
import numpy as np
import hmmlearn.hmm as hmm

model2 = hmm.MultinomialHMM(n_components=3, n_iter=20, tol=0.001, random_state=14)
# 给定的任意一个随机一个序列数据集
# 备注：序列与序列之间是独立互不相关的
X2 = np.array([
    [0, 1, 0, 0, 1], # 第一个序列: 白，黑，白，白，黑
    [0, 0, 0, 1, 1], # 第二个序列：白，白，白，黑，黑
    [1, 1, 0, 1, 0], # 第三个序列：黑，黑，白，黑，白
    [0, 1, 0, 1, 1], # 第四个序列：白，黑，白，黑，黑
    [0, 0, 0, 1, 0]  # 第五个序列：白，白，白，黑，白
])
X2 = X2.reshape((-1,1))
print(X2.shape)
lengths = [5 for i in range(X2.shape[0] // 5)]
model2.fit(X2, lengths=lengths)
print("输出根据数据训练出来的π")
print(model2.startprob_)
print("输出根据数据训练出来的A")
print(model2.transmat_)
print("输出根据数据训练出来的B")
print(model2.emissionprob_)

# 运行viterbi预测的问题
se = np.array([
    [0, 1, 0, 0, 1] # 白，黑，白，白，黑
]).T
print(model2.predict(se))
states2 = ['状态1', '状态2', '状态3']
logprod, box_index = model2.decode(se, algorithm='viterbi')
print("颜色:", end="")
print(" ".join(map(lambda t: obs[t], [0, 1, 0, 0, 1])))
print("盒子:", end="")
print(" ".join(map(lambda t: states2[t], box_index)))
print("概率值:", end="")
print(np.exp(logprod))

(25, 1)
输出根据数据训练出来的π
[  2.20057582e-01   7.79942405e-01   1.26182480e-08]
输出根据数据训练出来的A
[[  4.07750166e-02   2.37630989e-02   9.35461884e-01]
 [  3.97277636e-01   5.09576499e-04   6.02212788e-01]
 [  4.46265281e-02   9.31227409e-01   2.41460631e-02]]
输出根据数据训练出来的B
[[ 0.05543401  0.94456599]
 [ 0.99499589  0.00500411]
 [ 0.35467269  0.64532731]]
[1 0 2 1 2]
颜色:白球 黑球 白球 白球 黑球
盒子:状态2 状态1 状态3 状态2 状态3
概率值:0.0347913389725
